# Trying out GPT API for the first _real_ time.

This version is now outdated, so I will have to update the code.

## [See the new version here](#updated-for-v10)

In [46]:
%pip install -q openai==0.28
%pip install -q python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [47]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()

# api_key does not need to be specified (we use default var name)
openai.api_key = os.environ['OPENAI_API_KEY']

This is the first example of '[Function calling](https://platform.openai.com/docs/guides/gpt/function-calling)' provided by OpenAI:

In [1]:
import openai
import json

messages = [{"role": "user", "content": "What's the weather like in Boston?"}]

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response

print(run_conversation())


{
  "id": "chatcmpl-8Jo9QQwbPctEujY8nLRO13A59W1Wa",
  "object": "chat.completion",
  "created": 1699731704,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently sunny and windy with a temperature of 72 degrees."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 72,
    "completion_tokens": 17,
    "total_tokens": 89
  }
}


In [2]:
# creating GPT functions that can validate things for me from parent python script
def is_member_super_pal(member):
    # call parent function that dynamically validates current super pal
    if member == 'clippy':
        return f"No, {member} is not the super pal."
    else:
        return f"Yes, {member} is the super pal."

def run_conversation():
# creating my own chat role
    messages = [{"role": "system", "content": "You are a helpful assistant named Super Pal Bot. "\
                 "You help the members of a small Discord community called Bringus. "\
                 "Each week a new super pal is chosen at random from the list of Bringus members. "},
                 #"The current super pal has the ability to choose a new super pal or have one picked at random."},
                #{"role": "user", "content": "Is clippy currently pal?"}]
                {"role": "user", "content": "Is FilbertTheWizard currently pal?"}]
    
    functions = [
            {
                "name": "is_member_super_pal",
                "description": "Check if the given member is currently super pal",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "member": {
                            "type": "string",
                            "description": "The member name, e.g. clippy",
                        },
                    },
                    "required": ["member"],
                },
            }
        ]
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions
    )
    
    response_message = response["choices"][0]["message"]
    
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "is_member_super_pal": is_member_super_pal,
        }
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            member=function_args.get("member")
        )
    
        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages
        )  # get a new response from GPT where it can see the function response
        return second_response

print(run_conversation())

{
  "id": "chatcmpl-8Jo9ocOWZOQ4tM9Fl9mm4VzXTRPMy",
  "object": "chat.completion",
  "created": 1699731728,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Yes, FilbertTheWizard is currently the super pal. How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 102,
    "completion_tokens": 19,
    "total_tokens": 121
  }
}


# Updated for v1.0
v1.0.0 Migration Guide:
- [GitHub Discussion](https://github.com/openai/openai-python/discussions/742)
- [learn.microsoft.com docs](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/migration)

In [3]:
%pip install openai==1.2.3
%pip install python-dotenv

  Using cached openai-1.2.3-py3-none-any.whl.metadata (16 kB)
Using cached openai-1.2.3-py3-none-any.whl (220 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from openai import OpenAI
import json, os

# api_key does not need to be specified (we use default var name)
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

[OpenAI "chat" Python docs](https://platform.openai.com/docs/api-reference/chat/)


- Needed to update `functions` to [`tools`](https://platform.openai.com/docs/api-reference/chat/create?lang=python#chat-create-tools).
- [`function_call`](https://platform.openai.com/docs/api-reference/chat/create?lang=python#chat-create-function_call) was deprecated in favor of `tool_choice`.
- All the documentation and blog posts are innacurate now: [\[1\]](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models) [\[2\]](https://cookbook.openai.com/examples/how_to_call_functions_for_knowledge_retrieval) [\[3\]](https://platform.openai.com/docs/guides/function-calling)



I'm going to move from ChatCompletions to [Assistants API](https://platform.openai.com/docs/assistants/overview/agents)([Assistants](https://platform.openai.com/docs/api-reference/assistants)) since there is new support and the older approach is being deprecated.

>Text generation models are now capable of [JSON mode](https://platform.openai.com/docs/guides/text-generation/json-mode) and Reproducible outputs. We also launched the Assistants API to enable you to build agent-like experiences on top of our text-generation models. GPT-4 Turbo is available in preview by specifying gpt-4-1106-preview as the model name.

[api.md on openai-pythonGitHub repo](https://github.com/openai/openai-python/blob/baa9f07f5b503017ea6dae8f047d3cfc1ee22c37/api.md)

[Managing threads and messages - OpenAI documentation](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages)

I can even dig into [Run Steps](https://platform.openai.com/docs/api-reference/runs/listRunSteps) if I want to "explore  or display the inner workings of the Assistant and its tools".

In [2]:
# creating GPT functions that can validate things for me from parent python script
def is_member_super_pal(member):
    # call parent function that dynamically validates current super pal
    if member == 'clippy':
        return f"No, {member} is not the super pal."
    else:
        return f"Yes, {member} is the super pal."

# using tools for Assistant API
tools = [
        {
            "type": "function",
            "function": {
                "name": "is_member_super_pal",
                "description": "Check if the given member is currently super pal",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "member": {
                            "type": "string",
                            "description": "The member name, e.g. clippy",
                        },
                    },
                    "required": ["member"],
                }
            }
        }
    ]

# create assistant
assistant = client.beta.assistants.create(
    name="Super Pal Bot",
    instructions="You are a helpful assistant named Super Pal Bot. "\
                 "You help the members of a small Discord community called Bringus. "\
                 "Each week a new super pal is chosen at random from the list of Bringus members. ",
    tools=tools,
    model="gpt-3.5-turbo-1106"
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Is WilbertTheWizard currently super pal?"
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
    #instructions=Additional optional instructions.
)

In [3]:
# retrieve run status
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
# we expect to see run.status=="requires_action"
print(f"Run: {run}")
print(f"Run | Status: {run.status}")
print(f"Run | required_action: {run.required_action}")

# retrieve messages
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
# we expect to see only the user's message
print(f"Messages: {messages}")
print(f"Messages | Data: {messages.data}")


Run: Run(id='run_4PRfKmcUMH3fHH1sM9HqcTKg', assistant_id='asst_2WotmHtCBRarRd2L0mk3WLch', cancelled_at=None, completed_at=None, created_at=1699731769, expires_at=1699732369, failed_at=None, file_ids=[], instructions='You are a helpful assistant named Super Pal Bot. You help the members of a small Discord community called Bringus. Each week a new super pal is chosen at random from the list of Bringus members. ', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_EihcQOVWBqpZRhBaFbx1WinF', function=Function(arguments='{"member":"WilbertTheWizard"}', name='is_member_super_pal'), type='function')]), type='submit_tool_outputs'), started_at=1699731769, status='requires_action', thread_id='thread_o8g1ljkkpKUdNWg11quqnjcp', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='is_member_super_pal', parameters={'type': '

In [4]:
# array of available tools for this assistant
avail_tools = { "is_member_super_pal": is_member_super_pal }

# retrieve tool function name, arguments, and call id
tool_fn = avail_tools[run.required_action.
                      submit_tool_outputs.tool_calls[0].
                      function.name]
tool_args = json.loads(run.required_action.
                       submit_tool_outputs.tool_calls[0].
                       function.arguments)
tool_call_id = run.required_action.submit_tool_outputs.tool_calls[0].id

# call tool function and save output
tool_output = tool_fn(member=dict(tool_args).get('member'))

# submit tools output
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
      {
        "tool_call_id": tool_call_id,
        "output": tool_output,
      }
    ]
)

In [28]:
# retrieve run status
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
# we expect to see run.status=="completed"
print(f"Run Status | {run.status}")

# retrieve messages
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
# sort messages into chronological order
messages.data.sort(key=lambda x: x.created_at)
# we expect to see GPT's response
for message in messages.data:
    print(f"Message    | {message.role}: {message.content[0].text.value}")

Run Status | completed
Message    | user: Is WilbertTheWizard currently super pal?
Message    | assistant: Yes, WilbertTheWizard is currently the super pal! If you have any questions or need assistance, feel free to ask.
